<a href="https://colab.research.google.com/github/tmontaj/scripter/blob/main/Notebooks/Best_and_worst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Per batch fnc**

inputs: <br>
x, y, model, pre model, past model, npick

output: <br>
best npick and worest npick in a batch

**Per set fnc**

inputs:
dataset object, model, pre model, past model, npick

output:
best npick and worest npick in a batch

In [ ]:
inputs = [1,2,3]
logits = ['a', 'b', 'c']
loss   = [0.1, 0, 0.01]

zipped = zip(loss, logits, inputs)
sorted(zipped)

[(0, 'b', 2), (0.01, 'c', 3), (0.1, 'a', 1)]

In [ ]:
zipped

In [ ]:
def loss(x,y):
  return np.array([6,4])

def model(x):
  return x

In [ ]:
def best_and_worest_per_batch(x, y, model, npick, loss, pre_model=None, post_model=None):
  if pre_model:
    x = pre_model(x)
  
  logits = model(x)
  loss_  = loss(y, logits)

  if post_model:
    logits = post_model(logits)
  print("loss",loss_.shape)
  print("logits",logits.shape)
  print("y",y.shape)
  print("x",x.shape)
  loss_axis = np.argsort(loss_)

  all = zip(loss_[loss_axis], x[loss_axis], logits[loss_axis], y[loss_axis])

  n = npick if npick <= x.shape[0] else x.shape[0] 
  
  best   = loss_axis[:n]
  worest = np.flip(loss_axis[-n:])

  best   = (loss_[best], x[best], logits[best], y[best])
  worest = (loss_[worest], x[worest], logits[worest], y[worest])

  return best, worest

In [ ]:
import numpy as np


word1 = np.array([9,1,2,25,12,2,4,25,6,7])
word2 = np.array([0.9,0.3,.2,.25,.12,.2,.4,.25,.6,.8])

x = np.array([word2, word1])
y = np.array([word1, word2])


xx = best_and_worest_per_batch(x=x, y=y, model = model, npick = 1, loss = loss)

xx

loss (2,)
logits (2, 10)
y (2, 10)
x (2, 10)


((array([4]),
  array([[ 9.,  1.,  2., 25., 12.,  2.,  4., 25.,  6.,  7.]]),
  array([[ 9.,  1.,  2., 25., 12.,  2.,  4., 25.,  6.,  7.]]),
  array([[0.9 , 0.3 , 0.2 , 0.25, 0.12, 0.2 , 0.4 , 0.25, 0.6 , 0.8 ]])),
 (array([6]),
  array([[0.9 , 0.3 , 0.2 , 0.25, 0.12, 0.2 , 0.4 , 0.25, 0.6 , 0.8 ]]),
  array([[0.9 , 0.3 , 0.2 , 0.25, 0.12, 0.2 , 0.4 , 0.25, 0.6 , 0.8 ]]),
  array([[ 9.,  1.,  2., 25., 12.,  2.,  4., 25.,  6.,  7.]])))

In [ ]:
def mutual_best_and_worest(best_overall, worest_overall, best, worest, n):
  best_mutual = []
  worest_mutual = []

  for i in range(n):
    if best_overall[i][0] > best[i][0]:
      best_mutual.append(best_overall[i])
    else:
      best_mutual.append(best[i])
    
    if worest_overall[i][0] < worest[i][0]:
      worest_mutual.append(worest_overall[i])
    else:
      worest_mutual.append(worest[-i])

  return (best_mutual, worest_mutual)

In [ ]:
def best_and_worest_per_repleca(dataset, model, npick, global_npick, loss, pre_model=None, post_model=None):
  
  best_overall=None
  worest_overall=None

  for x,y in dataset:
    n = global_npick if global_npick <= x.shape[0] else x.shape[0]
    best, worest = best_and_worest_per_batch(x, y, model, npick, loss, pre_model, post_model)

    # best_overall, worest_overall = mutual_best_and_worest(best_overall, 
    #                                                       worest_overall, best, 
    #                                                       worest, n) if best_overall else (best, worest)

    if best_overall:
      best_overall, worest_overall = mutual_best_and_worest(best_overall, 
                                                          worest_overall, best, 
                                                          worest, n)
    else:
      best_overall, worest_overall = (best, worest)

  return best_overall, worest_overall